### Day 24

### Part 1:
- Build an Arithmetic Logic Unit
    - Integer variables w,x,y,z that start at 0
    - 6 instructions:
        - inp a : Read an input and store it as variable a
        - add a b: Add a + b and store it as variable a
        - mul a b: Multiply axb and store it as variable a
        - div a b: Do a//b and store it as variable a
        - mod a b: Do a % b and store it as variable a
        - eql a b: Store 1*(a==b) in variable a
    - Find the maximum possible 14 digit number that gives z=0 for the puzzle input
- Thoughts:
    - Won't try to brute force a 14 digit number
    - Can still build the ALU to check answers and better understand the problem
    - Should look at the puzzle input for patterns or simplifications

In [1]:
class ALU(object):
    def __init__(self,fname):
        """Loads a program from fname and sets it up."""
        with open(fname, "r") as f:
            data = f.read().splitlines()
        self.program = data
        
        self.vars = {
            "w":0,
            "x":0,
            "y":0,
            "z":0
        }
        
    def load_input(self,input_val:list):
        """Load an input into the ALU, ready to execute."""
        self.input = input_val
 
    def _get_input(self):
        """Get the next input value"""
        return self.input.pop(0)
    
    ### Instructions
    def inp(self,var):
        """Put the next input into variable var"""
        self.vars[var] = int(self._get_input())
    
    def add(self,var1,var2):
        """Do var1 = var1+var2"""
        if var2 in "wxyz":
            self.vars[var1] += self.vars[var2]
        else:
            self.vars[var1] += int(var2)
            
    def mul(self,var1,var2):
        """Do var1 = var1*var2"""
        if var2 in "wxyz":
            self.vars[var1] *= self.vars[var2]
        else:
            self.vars[var1] *= int(var2)
    
    def div(self,var1,var2):
        """Do var1 = var1//var2"""
        if var2 in "wxyz":
            self.vars[var1] = self.vars[var1]//self.vars[var2]
        else:
            self.vars[var1] = self.vars[var1]//int(var2)
    
    def mod(self,var1,var2):
        """Do var1 = var1 % var2"""
        if var2 in "wxyz":
            self.vars[var1] = self.vars[var1] % self.vars[var2]
        else:
            self.vars[var1] = self.vars[var1] % int(var2)
    
    def eql(self,var1,var2):
        """Replace var1 with 1 if var1==var2, 0 otherwise"""
        if var2 in "wxyz":
            self.vars[var1] = 1*(self.vars[var1] == self.vars[var2])
        else:
            self.vars[var1] = 1*(self.vars[var1] == int(var2))
     
    ###
    def run_program(self):
        for instruction in self.program:
            spl = instruction.split(" ")
            name = spl[0]
            var = spl[1:]
            
            if name == "inp":
                self.inp(var[0])
            elif name == "add":
                self.add(var[0], var[1])
            elif name == "mul":
                self.mul(var[0], var[1])
            elif name == "div":
                self.div(var[0], var[1])
            elif name == "mod":
                self.mod(var[0], var[1])
            elif name == "eql":
                self.eql(var[0], var[1])
            else:
                raise Exception("Unknown Instruction:"+str(name))
            

In [2]:
# Tests to check my ALU is working:
a = ALU("inputs/day24_test_input.dat")
a.load_input([1,2,3])

# Test input:
assert a._get_input() == 1

# Test loading a variable
a.inp("w")
a.inp("x")
assert a.vars["w"] == 2
assert a.vars["x"] == 3

# Test addition
a.add("w","5")
assert a.vars["w"] == 7
a.add("w","x")
assert a.vars["w"] == 10

# Test multiplication
a.mul("x","2")
assert a.vars["x"] == 6
a.mul("w","x")
assert a.vars["w"] == 60

# Test division
a.div("w","x")
assert a.vars["w"] == 10
a.div("w","2")
assert a.vars["w"] == 5

# Test modulo
a.mod("x","w")
assert a.vars["x"] == 1
a.mod("w","3")
assert a.vars["w"] == 2

# Test equal
a.eql("w","x")
assert a.vars["w"] == 0
a.eql("x","1")
assert a.vars["x"] == 1

# Run test program
a = ALU("inputs/day24_test_input.dat")
num = 27
a.load_input([num])
a.run_program()
num_bin = bin(num).replace("0b","")
should_be = {"w":int(num_bin[-4]), "x":int(num_bin[-3]), "y":int(num_bin[-2]), "z":int(num_bin[-1])}
assert a.vars == should_be

In [3]:
# Run puzzle input on one number
a = ALU("inputs/day24_input.dat")

test_num = 13579246899999
a.load_input([int(x) for x in str(test_num)])
a.run_program()
print(a.vars["z"])

4137701172


### Observations from the puzzle input (I did this by hand)
There are 2 kinds of code blocks:
1. **Those that read in the next number *w* and have //1**
    - These update z to z*26 + w + a (for some number a)
    - This always increases the size of z (and basically adds a new digit in base 26)
2. **Those that read in the next number *w* and have //26**
    - These update z to z//26 if (z%26)==b for some number b. (This basically removes a new digit in base 26)
    - Otherwise they increase the size of z. since these are the only way to decrease z, we need the other case to be true
- This looks like adding and removing digits in base 26.
- Code blocks type 1 add a digit that depends on w1 and blocks of type 2 remove a digit subject to a constraint between the numbers a,b,w1 and w2.
- So we can collect all of the constraints and then pick values that solve them
- Here's what the blocks do to z in base 26:
    1.  Add new digit num[0] + 12
    2.  Add new digit num[1] + 7
    3.  Add new digit num[2] + 1
    4.  Add new digit num[3] + 2
    5.  Remove last digit if it is num[4]+5, i.e. (num[3]+2)-5 is num[4]
    6.  Add new digit num[5]+15
    7.  Add new digit num[6]+11
    8.  Remove last digit if it is num[7]+13, i.e. (num[6]+11) -13 is num[7]
    9.  Remove last digit if it is num[8]+16, i.e. (num[5]+15) -16 is num[8]
    10. Remove last digit if it is num[9]+8, i.e. (num[2]+1) - 8 is num[9]
    11. Add new digit num[10]+2
    12. Remove last digit if it is num[11]+8, i.e. (num[10]+2) - 8 is num[11]
    13. Remove last digit if it is num[12]+0, i.e. (num[1]+7) is num[12]
    14. Remove last digit if it is num[13]+4, i.e. (num[0]+12) -4 is num[13]

In [4]:
### So, constraints:
### num[0]+8 = num[13]
### num[1]+7 = num[12]
### num[2]-7 = num[9]
### num[3]-3 = num[4]
### num[5]-1 = num[8]
### num[6]-2 = num[7]
### num[10]-6 = num[11]

### For the max number we need to maximize num[x] (on the left) while keeping num[x] and num[y] between 1 and 9
### So,
num = [0 for ix in range(14)]
num[0] = 1
num[1] = 2
num[2] = 9
num[3] = 9
num[5] = 9
num[6] = 9
num[10] = 9

num[13] = num[0]+8
num[12] = num[1]+7
num[9] = num[2]-7
num[4] = num[3]-3
num[8] = num[5]-1
num[7] = num[6]-2
num[11] = num[10]-6
print("Maximum number is","".join([str(x) for x in num]))

# Check it:
a = ALU("inputs/day24_input.dat")
a.load_input(num)
a.run_program()
print(a.vars["z"])

Maximum number is 12996997829399
0


### Part 2:
- Find the minimum number instead

In [5]:
### Again, constraints are:
### num[0]+8 = num[13]
### num[1]+7 = num[12]
### num[2]-7 = num[9]
### num[3]-3 = num[4]
### num[5]-1 = num[8]
### num[6]-2 = num[7]
### num[10]-6 = num[11]

### For the min number we need to minimize num[x] (on the left) while keeping num[x] and num[y] between 1 and 9
### So,

num = [0 for ix in range(14)]
num[0] = 1
num[1] = 1
num[2] = 8
num[3] = 4
num[5] = 2
num[6] = 3
num[10] = 7

num[13] = num[0]+8
num[12] = num[1]+7
num[9] = num[2]-7
num[4] = num[3]-3
num[8] = num[5]-1
num[7] = num[6]-2
num[11] = num[10]-6
print("Minimum number is","".join([str(x) for x in num]))

# Check it:
a = ALU("inputs/day24_input.dat")
a.load_input(num)
a.run_program()
print(a.vars["z"])


Minimum number is 11841231117189
0
